<a href="https://colab.research.google.com/github/MarceloClaro/QuantumDeepClassifier/blob/main/Rede_de_Tensores_Qu%C3%A2nticos_(FedQTNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das bibliotecas necessárias
!pip install qiskit
!pip install pennylane
!pip install tensorflow-quantum
!pip install matplotlib
!pip install pillow


In [ ]:
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o diretório de saída no Google Drive
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive

# Função para solicitar e verificar o caminho do arquivo ZIP
def get_zip_path():
    zip_path = input("Insira o caminho completo do arquivo melanomas.zip no seu Google Drive (ex: /content/drive/My Drive/melanomas.zip): ")
    if not os.path.exists(zip_path):
        print("Arquivo não encontrado! Verifique o caminho e tente novamente.")
        return None
    return zip_path

# Solicitar o caminho do arquivo ZIP
zip_path = get_zip_path()

if zip_path:
    # Extrair o arquivo ZIP
    extract_path = "/content/melanomas"
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Listar os arquivos extraídos
    extracted_files = []
    for root, dirs, files in os.walk(extract_path):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):  # Apenas imagens
                extracted_files.append(os.path.join(root, file))

    print(f"Número total de imagens extraídas: {len(extracted_files)}")
    print("Exemplos de arquivos extraídos:", extracted_files[:5])

    # Função para visualizar imagens por classe
    def visualize_images(files, n=5):
        """Visualizar as primeiras N imagens de cada classe"""
        classes = {}
        for file in files:
            class_name = os.path.basename(os.path.dirname(file))
            if class_name not in classes:
                classes[class_name] = []
            classes[class_name].append(file)

        for class_name, images in classes.items():
            print(f"Classe: {class_name} | Total de imagens: {len(images)}")
            plt.figure(figsize=(15, 5))
            plt.suptitle(f"Exemplos da classe: {class_name}")
            for i, img_path in enumerate(images[:n]):
                img = Image.open(img_path)
                plt.subplot(1, n, i + 1)
                plt.imshow(img)
                plt.axis("off")
                plt.title(f"{class_name} {i+1}")
            plt.show()

    # Visualizar as imagens
    visualize_images(extracted_files, n=5)

    # Função para redimensionar e salvar imagens
    def resize_and_save_images(files, image_size=(64, 64), save_dir=output_dir):
        """Redimensiona e salva as imagens no Google Drive organizadas por classe"""
        for file in files:
            class_name = os.path.basename(os.path.dirname(file))
            class_dir = os.path.join(save_dir, class_name)
            os.makedirs(class_dir, exist_ok=True)

            img = Image.open(file).convert('RGB')  # Garantir que a imagem tenha 3 canais
            img_resized = img.resize(image_size)
            save_path = os.path.join(class_dir, os.path.basename(file))
            img_resized.save(save_path)

    # Redimensionar e salvar as imagens
    resize_and_save_images(extracted_files, image_size=(64, 64), save_dir=output_dir)
    print(f"Imagens redimensionadas e salvas em: {output_dir}")

    # Função para criar um DataFrame com os dados
    def create_dataframe(save_dir=output_dir):
        """Cria um DataFrame com os dados das imagens e seus rótulos"""
        data = []
        labels = []
        for class_name in os.listdir(save_dir):
            class_path = os.path.join(save_dir, class_name)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_file)
                    img = Image.open(img_path).convert('RGB')
                    img_array = np.array(img).flatten() / 255.0  # Normalizar
                    data.append(img_array)
                    labels.append(class_name)
        df = pd.DataFrame(data)
        df['label'] = labels
        return df

    # Criar o DataFrame
    df = create_dataframe(save_dir=output_dir)
    print("Amostra do DataFrame:")
    print(df.head())
else:
    print("Encerrando o processo devido à falta do arquivo ZIP.")


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

if zip_path:
    # Separar features e labels
    X = df.drop('label', axis=1).values
    y = df['label'].values

    # Codificar os rótulos
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    print(f"Dados divididos em {X_train.shape[0]} amostras de treino e {X_test.shape[0]} amostras de teste.")
else:
    print("Não foi possível criar os DataFrames devido à falta do arquivo ZIP.")


In [ ]:
# 1. Importação das Bibliotecas Necessárias
import pennylane as qml
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as np  # Importar o NumPy do PennyLane
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from google.colab import drive

# 2. Montagem do Google Drive e Configuração do Ambiente
drive.mount('/content/drive')

# Definir o diretório de saída no Google Drive
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# 3. Definição do Dispositivo Quântico
n_qubits = 10
n_layers = 8  # Camadas do modelo quântico
dev = qml.device("default.qubit", wires=n_qubits)

# 4. Função de Embedding Quântico com Múltiplas Rotações
def data_embedding(features, wires):
    for i, wire in enumerate(wires):
        qml.RX(features[i], wires=wire)
        qml.RY(features[i], wires=wire)
        qml.RZ(features[i], wires=wire)

# 5. Modelo Quântico com StronglyEntanglingLayers
def quantum_model(weights, features):
    data_embedding(features, range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# 6. Definição do QNode
@qml.qnode(dev)
def circuit(weights, features):
    return quantum_model(weights, features)

# 7. Função de Custo com Regularização L2
def cost_with_regularization(weights, X, y, lambda_reg=0.05):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i])
        # Transformar a saída do PauliZ de [-1, 1] para [0, 1]
        pred_transformed = (pred + 1) / 2
        loss += (pred_transformed - y[i])**2
    reg_term = lambda_reg * np.sum(weights**2)
    return loss / len(X) + reg_term

# 8. Inicialização dos Pesos
weights_shape = (n_layers, n_qubits, 3)  # StronglyEntanglingLayers requer 3 parâmetros por qubit
weights = np.random.uniform(low=-0.1, high=0.1, size=weights_shape, requires_grad=True)

# 9. Configuração do Otimizador
opt = AdamOptimizer(stepsize=0.015)
steps = 100  # Número de iterações
early_stopping_patience = 3  # Paciência para Early Stopping
min_delta = 1e-4  # Melhoria mínima para considerar

# 10. Treinamento do Modelo Quântico
best_weights = None
best_test_cost = float('inf')
no_improvement_count = 0
train_costs = []
test_costs = []

for step in range(steps):
    weights = opt.step(lambda w: cost_with_regularization(w, X_train, y_train), weights)
    train_cost = cost_with_regularization(weights, X_train, y_train)
    test_cost = cost_with_regularization(weights, X_test, y_test)

    train_costs.append(train_cost)
    test_costs.append(test_cost)

    # Early Stopping
    if test_cost < best_test_cost - min_delta:
        best_test_cost = test_cost
        best_weights = weights
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_patience:
        print(f"Parada antecipada no passo {step}. Melhor custo no teste: {best_test_cost:.4f}")
        break

    if step % 10 == 0:
        print(f"Step {step}/{steps}: Train Cost = {train_cost:.4f} | Test Cost = {test_cost:.4f}")

# 11. Usar os Melhores Pesos Encontrados
weights = best_weights

# 12. Avaliação Final
final_train_cost = cost_with_regularization(weights, X_train, y_train)
final_test_cost = cost_with_regularization(weights, X_test, y_test)
print(f"Custo final no conjunto de treino: {final_train_cost:.4f}")
print(f"Custo final no conjunto de teste: {final_test_cost:.4f}")

# 13. Previsões
y_train_pred_probs = [(circuit(weights, x) + 1) / 2 for x in X_train]  # Probabilidades
y_test_pred_probs = [(circuit(weights, x) + 1) / 2 for x in X_test]

# Converter probabilidades para classes binárias com limiar 0.5
y_train_pred = [1 if prob > 0.5 else 0 for prob in y_train_pred_probs]
y_test_pred = [1 if prob > 0.5 else 0 for prob in y_test_pred_probs]

# 14. Relatório de Classificação
print("\nRelatório de Classificação (Treino):")
print(classification_report(y_train, y_train_pred, target_names=label_encoder.classes_, zero_division=1))
print("\nRelatório de Classificação (Teste):")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_, zero_division=1))

# 15. Métrica AUC-ROC
try:
    auc_train = roc_auc_score(y_train, y_train_pred_probs)
    auc_test = roc_auc_score(y_test, y_test_pred_probs)
    print(f"\nAUC-ROC (Treino): {auc_train:.4f}")
    print(f"AUC-ROC (Teste): {auc_test:.4f}")
except ValueError as e:
    print(f"Erro no cálculo do AUC-ROC: {e}")

# 16. Matriz de Confusão
print("\nMatriz de Confusão (Teste):")
print(confusion_matrix(y_test, y_test_pred))

# 17. Criação de DataFrames Separados para Treino e Teste
train_features = [x.tolist() for x in X_train]
test_features = [x.tolist() for x in X_test]

train_df = pd.DataFrame({
    "features": train_features,
    "labels": y_train,
    "predictions": y_train_pred,
    "predictions_probs": y_train_pred_probs,
    "residuals": [y - p for y, p in zip(y_train, y_train_pred)]
})

test_df = pd.DataFrame({
    "features": test_features,
    "labels": y_test,
    "predictions": y_test_pred,
    "predictions_probs": y_test_pred_probs,
    "residuals": [y - p for y, p in zip(y_test, y_test_pred)]
})

# 18. Salvando os Resultados no Google Drive
train_results_path = os.path.join(output_dir, "quantum_train_results.csv")
test_results_path = os.path.join(output_dir, "quantum_test_results.csv")

train_df.to_csv(train_results_path, index=False)
test_df.to_csv(test_results_path, index=False)

print(f"\nResultados de treino salvos em: {train_results_path}")
print(f"Resultados de teste salvos em: {test_results_path}")

# 19. Visualizações

# 1. Gráfico de Custo Durante o Treinamento
plt.figure(figsize=(10, 6))
plt.plot(range(len(train_costs)), train_costs, label="Custo de Treinamento", color="blue")
plt.plot(range(len(test_costs)), test_costs, label="Custo de Teste", color="orange")
plt.title("Evolução do Custo Durante o Treinamento")
plt.xlabel("Passos")
plt.ylabel("Custo")
plt.legend()
plt.savefig(os.path.join(output_dir, "training_costs.png"))
plt.show()

# 2. Gráfico de Dispersão (Previsão Probabilística vs Rótulo)
plt.figure(figsize=(10, 6))
plt.scatter(y_train, y_train_pred_probs, alpha=0.7, label="Treino", color="blue")
plt.scatter(y_test, y_test_pred_probs, alpha=0.7, label="Teste", color="orange")
plt.title("Dispersão: Previsão Probabilística vs Rótulo")
plt.xlabel("Rótulo Verdadeiro")
plt.ylabel("Probabilidade da Classe 1")
plt.legend()
plt.savefig(os.path.join(output_dir, "scatter_predictions.png"))
plt.show()

# 3. Histograma de Resíduos
plt.figure(figsize=(10, 6))
plt.hist(train_df["residuals"], bins=20, alpha=0.7, label="Treino", color="blue")
plt.hist(test_df["residuals"], bins=20, alpha=0.7, label="Teste", color="orange")
plt.title("Distribuição dos Resíduos")
plt.xlabel("Resíduo (Real - Predito)")
plt.ylabel("Frequência")
plt.legend()
plt.savefig(os.path.join(output_dir, "residuals_histogram.png"))
plt.show()


In [ ]:
# -*- coding: utf-8 -*-
"""
Redes de Tensores Quânticos Federados (FedQTNs) para Diagnóstico Médico
com Comparação de Modelo Clássico (Exemplo Simplificado)

1. Verificação de dados e organização
2. Data Augmentation e Redimensionamento de imagens
3. Pré-processamento (PCA, UMAP, t-SNE, Balanceamento de classes)
4. Aprendizado Federado (Modelo Quântico e Modelo Clássico)
5. Grad-CAM simplificado via tf-keras-vis para o modelo clássico (CNN)
6. Logging e um teste unitário de exemplo
"""

# =============================================================================
# 1. Instalação das Bibliotecas Necessárias (comente se já estiverem instaladas)
# =============================================================================

!pip install qiskit
!pip install pennylane
!pip install matplotlib
!pip install pillow
!pip install scikit-learn
!pip install albumentations
!pip install tensorflow
!pip install tf-keras-vis
!pip install Augmentor
!pip install umap-learn
!pip install diffprivlib
!pip install pytest

# =============================================================================
# 2. Importação das Bibliotecas Necessárias
# =============================================================================

import os
import zipfile
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from google.colab import drive

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import umap
import albumentations as A
import Augmentor

import pennylane as qml
from pennylane.optimize import AdamOptimizer

# Importar 'Gaussian' do diffprivlib
from diffprivlib.mechanisms import Gaussian

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.scores import CategoricalScore

import pytest  # Para testes unitários

# =============================================================================
# Configuração de Logging para Monitoramento
# =============================================================================

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =============================================================================
# 3. Montagem do Google Drive e Configuração de Diretórios
# =============================================================================

drive.mount('/content/drive')

base_dir = "/content/drive/My Drive/quantum_fedQTN"
melanomas_dir = os.path.join(base_dir, "MELANOMAS")
update_dir = os.path.join(base_dir, "ATUALIZACAO")
data_dir = os.path.join(base_dir, "DADOS")
os.makedirs(melanomas_dir, exist_ok=True)
os.makedirs(update_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

# =============================================================================
# 4. Função para Solicitar e Verificar o Caminho do Arquivo ZIP
# =============================================================================

def get_zip_path():
    """Solicita ao usuário o caminho completo do arquivo melanomas.zip no seu Google Drive."""
    zip_path = input(
        "Insira o caminho completo do arquivo melanomas.zip no seu Google Drive: "
    )
    if not os.path.exists(zip_path):
        print("Arquivo não encontrado! Verifique o caminho e tente novamente.")
        return None
    return zip_path

# =============================================================================
# 5. Solicitar o Caminho do Arquivo ZIP
# =============================================================================

zip_path = get_zip_path()

# =============================================================================
# 6. Verificação de Organização dos Dados
# =============================================================================

def verify_data_organization(directory):
    """
    Verifica se os dados estão organizados em subpastas por classe dentro do 'directory'.
    Cada subpasta deve conter imagens (.jpg, .png, etc.).
    """
    classes = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    if not classes:
        logging.error(f"Nenhuma subpasta encontrada em {directory}. Verifique a organização dos dados.")
        return False
    for cls in classes:
        cls_dir = os.path.join(directory, cls)
        images = [f for f in os.listdir(cls_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        if not images:
            logging.error(f"Nenhuma imagem encontrada na subpasta {cls_dir}.")
            return False
    logging.info("Verificação de organização de dados concluída com sucesso.")
    return True

# =============================================================================
# 7. Funções Auxiliares para Visualização e Data Augmentation
# =============================================================================

def visualize_images(files, n=5):
    """Visualiza as primeiras N imagens de cada classe para monitorar o dataset."""
    classes = {}
    for file in files:
        class_name = os.path.basename(os.path.dirname(file))
        if class_name not in classes:
            classes[class_name] = []
        classes[class_name].append(file)

    for class_name, images in classes.items():
        logging.info(f"Classe: {class_name} | Total de imagens: {len(images)}")
        plt.figure(figsize=(15, 5))
        plt.suptitle(f"Exemplos da classe: {class_name}")
        for i, img_path in enumerate(images[:n]):
            img = Image.open(img_path)
            plt.subplot(1, n, i + 1)
            plt.imshow(img)
            plt.axis("off")
            plt.title(f"{class_name} {i+1}")
        plt.show()

def augment_images(files, save_dir):
    """
    Aplica transformações de Data Augmentation usando Albumentations e salva as imagens.
    (Removendo ElasticTransform e IAAPiecewiseAffine)
    """
    albumentations_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.Rotate(limit=45, p=0.7),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.RandomResizedCrop(height=64, width=64, scale=(0.8, 1.0), ratio=(0.9, 1.1), p=0.5),
        A.Blur(blur_limit=3, p=0.3),
        A.HueSaturationValue(p=0.3),
        A.RandomGamma(p=0.3)
    ])

    for file in files:
        img = Image.open(file).convert('RGB')
        img_array = np.array(img)
        # Gerar 2 imagens com Albumentations
        for idx in range(2):
            aug_img = albumentations_transform(image=img_array)['image']
            aug_img_pil = Image.fromarray(aug_img)
            aug_filename = os.path.splitext(os.path.basename(file))[0] + f"_alb_aug{idx}.jpg"
            aug_img_pil.save(os.path.join(save_dir, aug_filename))
            logging.info(f"Imagem augmentada salva em: {os.path.join(save_dir, aug_filename)}")

# =============================================================================
# 8. Função para Redimensionar e Salvar Imagens
# =============================================================================

def resize_and_save_images(files, image_size=(64, 64), save_dir=None):
    if save_dir is None:
        save_dir = data_dir

    for file in files:
        class_name = os.path.basename(os.path.dirname(file))
        class_dir = os.path.join(save_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

        img = Image.open(file).convert('RGB')
        img_resized = img.resize(image_size)
        save_path = os.path.join(class_dir, os.path.basename(file))
        img_resized.save(save_path)
        logging.info(f"Imagem redimensionada salva em: {save_path}")

# =============================================================================
# 9. Função para Criar um DataFrame com os Dados
# =============================================================================

def create_dataframe(save_dir=None):
    if save_dir is None:
        save_dir = data_dir

    data = []
    labels = []
    for class_name in os.listdir(save_dir):
        class_path = os.path.join(save_dir, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(class_path, img_file)
                    img = Image.open(img_path).convert('RGB')
                    img_array = np.array(img).flatten() / 255.0
                    data.append(img_array)
                    labels.append(class_name)
    df = pd.DataFrame(data)
    df['label'] = labels
    return df

# =============================================================================
# 10. Carregamento e Processamento do ZIP se Disponível
# =============================================================================

if zip_path:
    # Extrair o Arquivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(melanomas_dir)
    logging.info("Arquivo ZIP extraído com sucesso.")

    # Verificar a organização dos dados
    if not verify_data_organization(melanomas_dir):
        raise Exception("Organização de dados incorreta. Verifique o diretório de imagens.")

    # Listar arquivos extraídos
    extracted_files = []
    for root, dirs, files in os.walk(melanomas_dir):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                extracted_files.append(os.path.join(root, file))

    logging.info(f"Número total de imagens extraídas: {len(extracted_files)}")
    logging.info(f"Exemplos de arquivos extraídos: {extracted_files[:5]}")

    # Visualizar algumas imagens
    visualize_images(extracted_files, n=5)

    # Aplicar Data Augmentation
    augment_images(extracted_files, save_dir=melanomas_dir)
    logging.info("Data Augmentation concluída.")

    # Atualizar lista de imagens após augmentação
    augmented_files = []
    for root, dirs, files in os.walk(melanomas_dir):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                augmented_files.append(os.path.join(root, file))

    logging.info(f"Número total de imagens após Data Augmentation: {len(augmented_files)}")

    # Redimensionar e salvar as imagens
    resize_and_save_images(augmented_files, image_size=(64, 64), save_dir=data_dir)
    logging.info(f"Imagens redimensionadas e salvas em: {data_dir}")

    # Criar DataFrame final
    df = create_dataframe(save_dir=data_dir)
    logging.info("Amostra do DataFrame:")
    print(df.head())

    # =============================================================================
    # 11. Pré-processamento (PCA, UMAP, t-SNE, Balanceamento)
    # =============================================================================

    X = df.drop('label', axis=1).values
    y = df['label'].values

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    def apply_pca(X, variance_threshold=0.95):
        pca = PCA(n_components=variance_threshold, svd_solver='full')
        X_pca = pca.fit_transform(X)
        logging.info(f"PCA -> {X_pca.shape[1]} comps explicando {pca.explained_variance_ratio_.sum()*100:.2f}% da variância.")
        return X_pca, pca

    variance_thresholds = [0.90, 0.95, 0.99]
    pca_results = {}
    for threshold in variance_thresholds:
        logging.info(f"\nAplicando PCA c/ threshold de variância: {threshold}")
        X_pca_tmp, pca_obj = apply_pca(X, variance_threshold=threshold)
        pca_results[threshold] = (X_pca_tmp, pca_obj)

    chosen_threshold = 0.95
    X_pca, pca = pca_results[chosen_threshold]

    def plot_pca_components(pca, num_components=10):
        plt.figure(figsize=(10,6))
        plt.bar(range(1, num_components+1), pca.explained_variance_ratio_[:num_components]*100)
        plt.xlabel('Componentes Principais')
        plt.ylabel('Variância Explicada (%)')
        plt.title('Importância dos Componentes Principais no PCA')
        plt.show()

    plot_pca_components(pca, num_components=10)

    def apply_umap_tsne(X, n_components=2):
        umap_model = umap.UMAP(n_components=n_components, random_state=42)
        X_umap = umap_model.fit_transform(X)
        tsne_model = TSNE(n_components=n_components, random_state=42)
        X_tsne = tsne_model.fit_transform(X)
        return X_umap, X_tsne

    X_umap, X_tsne = apply_umap_tsne(X_pca)

    def plot_umap(X_umap, y_encoded, label_encoder):
        plt.figure(figsize=(10,6))
        for class_idx in np.unique(y_encoded):
            plt.scatter(X_umap[y_encoded == class_idx, 0],
                        X_umap[y_encoded == class_idx, 1],
                        label=label_encoder.inverse_transform([class_idx])[0],
                        alpha=0.5)
        plt.title('Projeção UMAP dos Dados')
        plt.xlabel('UMAP 1')
        plt.ylabel('UMAP 2')
        plt.legend()
        plt.show()

    def plot_tsne(X_tsne, y_encoded, label_encoder):
        plt.figure(figsize=(10,6))
        for class_idx in np.unique(y_encoded):
            plt.scatter(X_tsne[y_encoded == class_idx, 0],
                        X_tsne[y_encoded == class_idx, 1],
                        label=label_encoder.inverse_transform([class_idx])[0],
                        alpha=0.5)
        plt.title('Projeção t-SNE dos Dados')
        plt.xlabel('t-SNE 1')
        plt.ylabel('t-SNE 2')
        plt.legend()
        plt.show()

    plot_umap(X_umap, y_encoded, label_encoder)
    plot_tsne(X_tsne, y_encoded, label_encoder)

    df_pca = pd.DataFrame(X_pca)
    df_pca['label'] = y_encoded
    class_counts = df_pca['label'].value_counts()
    min_count = class_counts.min()

    df_balanced = pd.DataFrame()
    for cls_id in class_counts.index:
        df_cls = df_pca[df_pca['label'] == cls_id]
        df_cls_resampled = resample(df_cls, replace=True, n_samples=min_count, random_state=42)
        df_balanced = pd.concat([df_balanced, df_cls_resampled])

    X_balanced = df_balanced.drop('label', axis=1).values
    y_balanced = df_balanced['label'].values

    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
    )
    logging.info(f"Dados de treino: {X_train_full.shape[0]}, teste: {X_test.shape[0]}.")

    num_clients = 4
    client_data = []
    X_split = np.array_split(X_train_full, num_clients)
    y_split = np.array_split(y_train_full, num_clients)

    for i in range(num_clients):
        client_data.append((X_split[i], y_split[i]))

    for i in range(num_clients):
        X_tr, X_val, y_tr, y_val = train_test_split(
            client_data[i][0], client_data[i][1], test_size=0.1,
            random_state=42, stratify=client_data[i][1]
        )
        client_data[i] = (X_tr, y_tr, X_val, y_val)

    logging.info(f"Cliente 1 -> Treino: {client_data[0][0].shape[0]}, Val: {client_data[0][2].shape[0]}")

    # =============================================================================
    # 12. Definição do Modelo Quântico (MERA) e Treinamento Federado
    # =============================================================================

    def MERA(weights, features, quantum_gates='RY'):
        wires = list(range(len(features)))
        for layer in range(len(weights)):
            for i in range(0, len(wires), 2):
                if i+1 < len(wires):
                    if quantum_gates == 'RY':
                        qml.RY(weights[layer][i][0], wires=wires[i])
                        qml.RY(weights[layer][i+1][0], wires=wires[i+1])
                    elif quantum_gates == 'RX':
                        qml.RX(weights[layer][i][0], wires=wires[i])
                        qml.RX(weights[layer][i+1][0], wires=wires[i+1])
                    qml.CNOT(wires=[wires[i], wires[i+1]])
                    if quantum_gates == 'RY':
                        qml.RY(weights[layer][i][1], wires=wires[i])
                        qml.RY(weights[layer][i+1][1], wires=wires[i+1])
                    elif quantum_gates == 'RX':
                        qml.RX(weights[layer][i][1], wires=wires[i])
                        qml.RX(weights[layer][i+1][1], wires=wires[i+1])
            new_wires = []
            for j in range(0, len(wires), 2):
                if j+1 < len(wires):
                    new_wires.append(j//2)
            wires = new_wires
        return qml.expval(qml.PauliZ(0))

    n_qubits = 8
    n_layers = 3
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit_mera(weights, features, quantum_gates='RY'):
        for i in range(n_qubits):
            qml.RX(features[i], wires=i)
            qml.RY(features[i], wires=i)
            qml.RZ(features[i], wires=i)
        return MERA(weights, features, quantum_gates)

    def cost_with_regularization(weights, X, y, lambda_reg=0.05, quantum_gates='RY'):
        loss = 0
        for i in range(len(X)):
            pred = circuit_mera(weights, X[i], quantum_gates)
            pred_transformed = (pred + 1)/2
            loss += (pred_transformed - y[i])**2
        reg_term = lambda_reg * np.sum(weights**2)
        return loss / len(X) + reg_term

    weights_shape = (n_layers, n_qubits, 2)
    weights = np.random.uniform(low=-0.1, high=0.1, size=weights_shape, requires_grad=True)

    def federated_averaging(global_weights, client_weights):
        avg_weights = copy.deepcopy(global_weights)
        for layer in range(len(global_weights)):
            for qubit in range(len(global_weights[layer])):
                avg_weights[layer][qubit] = np.mean(
                    [client_weights[c][layer][qubit] for c in range(len(client_weights))],
                    axis=0
                )
        return avg_weights

    opt = AdamOptimizer(stepsize=0.01)
    rounds = 5
    local_steps = 20
    lambda_reg = 0.05
    dp_noise = 0.05

    def train_federated(global_weights, client_data, rounds=5, local_steps=20,
                        lambda_reg=0.05, dp_noise=0.05, quantum_gates='RY'):
        privacy_metrics = {}
        for r in range(rounds):
            logging.info(f"\n--- Rodada Federada {r+1}/{rounds} ---")
            client_updates = []
            epsilons = []
            deltas = []
            for c in range(len(client_data)):
                logging.info(f"Treinando Cliente {c+1} (Quântico)")
                local_weights = copy.deepcopy(global_weights)
                X_train_q, y_train_q, X_val_q, y_val_q = client_data[c]

                best_local_weights = None
                best_val_cost = float('inf')
                no_improvement = 0
                for step in range(local_steps):
                    local_weights = opt.step(
                        lambda w: cost_with_regularization(w, X_train_q, y_train_q, lambda_reg, quantum_gates),
                        local_weights
                    )
                    val_cost = cost_with_regularization(local_weights, X_val_q, y_val_q, lambda_reg, quantum_gates)
                    if val_cost < best_val_cost - 1e-4:
                        best_val_cost = val_cost
                        best_local_weights = copy.deepcopy(local_weights)
                        no_improvement = 0
                    else:
                        no_improvement += 1
                    if no_improvement >= 10:
                        logging.info(f"Parada antecipada no cliente {c+1}, passo {step+1}.")
                        break

                if best_local_weights is None:
                    best_local_weights = local_weights

                mechanism = Gaussian()
                mechanism.set_variance(dp_noise**2)
                noisy_weights = mechanism(best_local_weights)
                client_updates.append(noisy_weights)

                epsilon = dp_noise
                delta = 1e-5
                epsilons.append(epsilon)
                deltas.append(delta)

                client_update_path = os.path.join(update_dir, f"client_{c+1}_update_round_{r+1}.npy")
                np.save(client_update_path, noisy_weights)
                logging.info(f"Atualização do Cliente {c+1} salva em: {client_update_path}")

            global_weights = federated_averaging(global_weights, client_updates)
            logging.info(f"Agregação concluída na rodada {r+1} (Quântico)")

            global_weights_path = os.path.join(update_dir, f"global_weights_round_{r+1}.npy")
            np.save(global_weights_path, global_weights)
            logging.info(f"Pesos globais após a rodada {r+1} salvos em: {global_weights_path}")

            privacy_metrics[r+1] = {
                'epsilon': np.mean(epsilons),
                'delta': np.mean(deltas)
            }
            logging.info(f"Rodada {r+1} - ε={np.mean(epsilons):.4f}, δ={np.mean(deltas):.4f}")

        return global_weights, privacy_metrics

    global_weights = copy.deepcopy(weights)
    global_weights, privacy_metrics = train_federated(
        global_weights, client_data,
        rounds=rounds, local_steps=local_steps,
        lambda_reg=lambda_reg, dp_noise=dp_noise, quantum_gates='RY'
    )

    def evaluate_model(weights, client_data, label_encoder, save_dir=data_dir, quantum_gates='RY'):
        for c in range(len(client_data)):
            X_tr_q, y_tr_q, X_val_q, y_val_q = client_data[c]
            X_test_q = X_val_q
            y_test_q = y_val_q

            y_test_pred_probs = [(circuit_mera(weights, x, quantum_gates) + 1)/2 for x in X_test_q]
            y_test_pred = [1 if prob > 0.5 else 0 for prob in y_test_pred_probs]

            logging.info(f"\nCliente {c+1} - Modelo Quântico")
            print(classification_report(y_test_q, y_test_pred, target_names=label_encoder.classes_, zero_division=1))
            try:
                auc_test_q = roc_auc_score(y_test_q, y_test_pred_probs)
                logging.info(f"AUC-ROC (Quântico)={auc_test_q:.4f}")
            except ValueError as e:
                logging.error(f"Erro no ROC Quântico: {e}")
            print(confusion_matrix(y_test_q, y_test_pred))

            test_df_q = pd.DataFrame({
                "features": [x.tolist() for x in X_test_q],
                "labels": y_test_q,
                "predictions": y_test_pred,
                "predictions_probs": y_test_pred_probs,
                "residuals": [y - p for y, p in zip(y_test_q, y_test_pred)]
            })
            test_res_path = os.path.join(save_dir, f"client_{c+1}_test_results_quantum.csv")
            test_df_q.to_csv(test_res_path, index=False)
            logging.info(f"Teste Quântico Cliente {c+1} salvo em: {test_res_path}")

    evaluate_model(global_weights, client_data, label_encoder)

    final_q_weights_path = os.path.join(update_dir, "global_weights_final.npy")
    np.save(final_q_weights_path, global_weights)
    logging.info(f"Pesos globais finais (Quântico) em: {final_q_weights_path}")

    # =============================================================================
    # 13. Modelo Clássico (CNN) e Aprendizado Federado
    # =============================================================================

    def create_cnn_model(input_shape=(64,), num_classes=2):
        model = models.Sequential()
        model.add(layers.Reshape((8, 8, 1), input_shape=input_shape))
        model.add(layers.Conv2D(32, (3,3), activation='relu', name='conv2d_1'))
        model.add(layers.MaxPooling2D((2,2), name='maxpool_1'))
        model.add(layers.Conv2D(64, (3,3), activation='relu', name='conv2d_2'))
        model.add(layers.MaxPooling2D((2,2), name='maxpool_2'))
        model.add(layers.Conv2D(64, (3,3), activation='relu', name='conv2d_3'))
        model.add(layers.Flatten(name='flatten'))
        model.add(layers.Dense(64, activation='relu', name='dense_1'))
        model.add(layers.Dense(num_classes, activation='softmax', name='output'))
        return model

    num_classes = len(label_encoder.classes_)

    def prepare_cnn_data(X, y, num_classes):
        X = X.reshape(-1, 8, 8, 1).astype('float32')
        y = tf.keras.utils.to_categorical(y, num_classes)
        return X, y

    X_train_full_cnn, y_train_full_cnn = prepare_cnn_data(X_train_full, y_train_full, num_classes)
    X_test_cnn, y_test_cnn = prepare_cnn_data(X_test, y_test, num_classes)

    client_data_cnn = []
    X_split_cnn = np.array_split(X_train_full_cnn, num_clients)
    y_split_cnn = np.array_split(y_train_full_cnn, num_clients)
    for i in range(num_clients):
        client_data_cnn.append((X_split_cnn[i], y_split_cnn[i]))

    for i in range(num_clients):
        X_tr_cnn, X_val_cnn, y_tr_cnn, y_val_cnn = train_test_split(
            client_data_cnn[i][0], client_data_cnn[i][1],
            test_size=0.1, random_state=42,
            stratify=np.argmax(client_data_cnn[i][1], axis=1)
        )
        client_data_cnn[i] = (X_tr_cnn, y_tr_cnn, X_val_cnn, y_val_cnn)

    logging.info(f"Cliente 1 (CNN) -> Treino: {client_data_cnn[0][0].shape[0]}, Val: {client_data_cnn[0][2].shape[0]}")

    cnn_model = create_cnn_model((64,), num_classes)
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train_federated_cnn(global_weights_cnn, client_data_cnn, rounds=5, epochs=5, dp_noise=0.05):
        privacy_metrics_cnn = {}
        for r in range(rounds):
            logging.info(f"\n--- Rodada Federada {r+1}/{rounds} - CNN ---")
            client_updates_cnn = []
            epsilons_cnn = []
            deltas_cnn = []
            for c in range(len(client_data_cnn)):
                logging.info(f"Treinando Cliente {c+1} (CNN)")
                local_model = create_cnn_model((64,), num_classes)
                local_model.set_weights(global_weights_cnn)

                X_tr_cnn, y_tr_cnn, X_val_cnn, y_val_cnn = client_data_cnn[c]
                early_stop = EarlyStopping(monitor='val_loss', patience=5, min_delta=1e-4, restore_best_weights=True)
                history = local_model.fit(
                    X_tr_cnn, y_tr_cnn,
                    epochs=epochs, batch_size=32,
                    validation_data=(X_val_cnn, y_val_cnn),
                    callbacks=[early_stop], verbose=0
                )
                logging.info(f"Cliente {c+1} - CNN - final val_loss={history.history['val_loss'][-1]:.4f}")

                trained_weights = local_model.get_weights()
                noisy_weights_cnn = []
                for layer_w in trained_weights:
                    noise = np.random.normal(0, dp_noise, layer_w.shape)
                    noisy_weights_cnn.append(layer_w + noise)
                client_updates_cnn.append(noisy_weights_cnn)

                epsilon_cnn = dp_noise
                delta_cnn = 1e-5
                epsilons_cnn.append(epsilon_cnn)
                deltas_cnn.append(delta_cnn)

                client_path_cnn = os.path.join(update_dir, f"client_{c+1}_cnn_update_round_{r+1}.npy")
                np.save(client_path_cnn, noisy_weights_cnn)
                logging.info(f"Atualização CNN Cliente {c+1} salva em: {client_path_cnn}")

            # Agregação
            aggregated_weights_cnn = []
            for layer_idx in range(len(global_weights_cnn)):
                layer_agg = []
                for w in range(len(client_updates_cnn[0][layer_idx])):
                    w_layer = [client_updates_cnn[c][layer_idx][w] for c in range(len(client_updates_cnn))]
                    layer_agg.append(np.mean(w_layer, axis=0))
                aggregated_weights_cnn.append(layer_agg)

            global_weights_cnn = aggregated_weights_cnn
            logging.info(f"Agregação concluída na rodada {r+1} - CNN")

            global_weights_path_cnn = os.path.join(update_dir, f"global_weights_cnn_round_{r+1}.npy")
            np.save(global_weights_path_cnn, global_weights_cnn)
            logging.info(f"Pesos globais salvos em: {global_weights_path_cnn}")

            privacy_metrics_cnn[r+1] = {'epsilon': np.mean(epsilons_cnn), 'delta': np.mean(deltas_cnn)}
            logging.info(f"Métricas Privacidade (CNN) R{r+1}: ε={np.mean(epsilons_cnn):.4f}, δ={np.mean(deltas_cnn):.4f}")

        return global_weights_cnn, privacy_metrics_cnn

    global_weights_cnn = cnn_model.get_weights()
    global_weights_cnn, privacy_metrics_cnn = train_federated_cnn(
        global_weights_cnn, client_data_cnn,
        rounds=5, epochs=5, dp_noise=dp_noise
    )

    def evaluate_model_cnn(global_weights_cnn, client_data_cnn, label_encoder, save_dir=data_dir):
        for c in range(len(client_data_cnn)):
            X_tr_cnn, y_tr_cnn, X_val_cnn, y_val_cnn = client_data_cnn[c]
            model_cnn_eval = create_cnn_model((64,), num_classes)
            model_cnn_eval.set_weights(global_weights_cnn)

            y_pred_probs_cnn = model_cnn_eval.predict(X_val_cnn).flatten()
            y_pred_cnn = np.argmax(model_cnn_eval.predict(X_val_cnn), axis=1)
            logging.info(f"\nCliente {c+1} - CNN")
            print(classification_report(np.argmax(y_val_cnn, axis=1), y_pred_cnn,
                                        target_names=label_encoder.classes_, zero_division=1))
            try:
                auc_test_cnn = roc_auc_score(np.argmax(y_val_cnn, axis=1), y_pred_probs_cnn)
                logging.info(f"AUC-ROC (CNN)={auc_test_cnn:.4f}")
            except ValueError as e:
                logging.error(f"Erro no AUC (CNN): {e}")

            print(confusion_matrix(np.argmax(y_val_cnn, axis=1), y_pred_cnn))

            test_features_cnn = [x.tolist() for x in X_val_cnn]
            test_df_cnn = pd.DataFrame({
                "features": test_features_cnn,
                "labels": np.argmax(y_val_cnn, axis=1),
                "predictions": y_pred_cnn,
                "predictions_probs": y_pred_probs_cnn,
                "residuals": [y - p for y, p in zip(np.argmax(y_val_cnn, axis=1), y_pred_cnn)]
            })
            csv_path_cnn = os.path.join(save_dir, f"client_{c+1}_test_results_cnn.csv")
            test_df_cnn.to_csv(csv_path_cnn, index=False)
            logging.info(f"Resultados CNN salvos em: {csv_path_cnn}")

    evaluate_model_cnn(global_weights_cnn, client_data_cnn, label_encoder)

    final_cnn_path = os.path.join(update_dir, "global_weights_final_cnn.npy")
    np.save(final_cnn_path, global_weights_cnn)
    logging.info(f"Pesos globais finais do CNN salvos em: {final_cnn_path}")

    # =============================================================================
    # 13. Visualizações Adicionais (Scatter / Histogram Residuals)
    # =============================================================================

    def plot_scatter_predictions_quantum(client_data, weights, label_encoder, save_dir=data_dir, quantum_gates='RY'):
        for c in range(len(client_data)):
            X_tr, y_tr, X_val, y_val = client_data[c]
            y_val_pred_probs = [(circuit_mera(weights, x, quantum_gates)+1)/2 for x in X_val]
            y_val_pred = [1 if prob>0.5 else 0 for prob in y_val_pred_probs]
            plt.figure(figsize=(10,6))
            plt.scatter(y_val, y_val_pred_probs, alpha=0.7, label=f"Cliente {c+1}")
            plt.title(f"Scatter: Prob vs Rótulo (Cliente {c+1}) - Quântico")
            plt.xlabel("Rótulo Verdadeiro")
            plt.ylabel("Prob (Classe=1)")
            plt.legend()
            sp_path = os.path.join(save_dir, f"scatter_quantum_client_{c+1}.png")
            plt.savefig(sp_path)
            plt.show()
            logging.info(f"Scatter Quântico salvo em: {sp_path}")

    def plot_scatter_predictions_cnn(client_data_cnn, weights_cnn, label_encoder, save_dir=data_dir):
        for c in range(len(client_data_cnn)):
            X_tr, y_tr, X_val, y_val = client_data_cnn[c]
            cnn_eval = create_cnn_model((64,), num_classes)
            cnn_eval.set_weights(weights_cnn)
            y_val_pred_probs_cnn = cnn_eval.predict(X_val).flatten()
            y_val_pred_cnn = np.argmax(cnn_eval.predict(X_val), axis=1)

            plt.figure(figsize=(10,6))
            plt.scatter(np.argmax(y_val, axis=1), y_val_pred_probs_cnn, alpha=0.7, label=f"Cliente {c+1}")
            plt.title(f"Scatter: Prob vs Rótulo (Cliente {c+1}) - CNN")
            plt.xlabel("Rótulo Verdadeiro")
            plt.ylabel("Prob (Classe=1)")
            plt.legend()
            sp_path_cnn = os.path.join(save_dir, f"scatter_cnn_client_{c+1}.png")
            plt.savefig(sp_path_cnn)
            plt.show()
            logging.info(f"Scatter CNN salvo em: {sp_path_cnn}")

    def plot_residuals_histogram_quantum(client_data, weights, save_dir=data_dir, quantum_gates='RY'):
        for c in range(len(client_data)):
            X_tr, y_tr, X_val, y_val = client_data[c]
            y_val_probs = [(circuit_mera(weights, x, quantum_gates)+1)/2 for x in X_val]
            y_val_pred = [1 if prob>0.5 else 0 for prob in y_val_probs]
            residuals = [y - p for y, p in zip(y_val, y_val_pred)]
            plt.figure(figsize=(10,6))
            plt.hist(residuals, bins=20, alpha=0.7)
            plt.title(f"Hist Resíduos Cliente {c+1} - Quântico")
            plt.xlabel("Resíduo (Real - Predito)")
            plt.ylabel("Frequência")
            hr_path = os.path.join(save_dir, f"hist_res_quantum_client_{c+1}.png")
            plt.savefig(hr_path)
            plt.show()
            logging.info(f"Hist de Resíduos Quântico salvo em: {hr_path}")

    def plot_residuals_histogram_cnn(client_data_cnn, weights_cnn, save_dir=data_dir):
        for c in range(len(client_data_cnn)):
            X_tr, y_tr, X_val, y_val = client_data_cnn[c]
            model_cnn_eval = create_cnn_model((64,), num_classes)
            model_cnn_eval.set_weights(weights_cnn)
            y_val_pred_cnn = np.argmax(model_cnn_eval.predict(X_val), axis=1)
            y_val_labels = np.argmax(y_val, axis=1)
            residuals_cnn = [y - p for y, p in zip(y_val_labels, y_val_pred_cnn)]
            plt.figure(figsize=(10,6))
            plt.hist(residuals_cnn, bins=20, alpha=0.7)
            plt.title(f"Hist Resíduos Cliente {c+1} - CNN")
            plt.xlabel("Resíduo (Real - Predito)")
            plt.ylabel("Frequência")
            hr_cnn_path = os.path.join(save_dir, f"hist_res_cnn_client_{c+1}.png")
            plt.savefig(hr_cnn_path)
            plt.show()
            logging.info(f"Hist de Resíduos CNN salvo em: {hr_cnn_path}")

    plot_scatter_predictions_quantum(client_data, global_weights, label_encoder, save_dir=data_dir)
    plot_scatter_predictions_cnn(client_data_cnn, global_weights_cnn, label_encoder, save_dir=data_dir)
    plot_residuals_histogram_quantum(client_data, global_weights, save_dir=data_dir)
    plot_residuals_histogram_cnn(client_data_cnn, global_weights_cnn, save_dir=data_dir)

    # =============================================================================
    # 14. Classificação de Imagem do Usuário c/ Grad-CAM Simplificado (CNN)
    # =============================================================================

    def classify_user_image_both_models(model_weights_quantum, pca, label_encoder,
                                        global_weights_cnn, save_dir=data_dir,
                                        quantum_gates='RY'):
        from google.colab import files
        uploaded = files.upload()
        if not uploaded:
            print("Nenhuma imagem foi carregada.")
            return

        for fn in uploaded.keys():
            img_path = fn
            img = Image.open(img_path).convert('RGB')
            img_resized = img.resize((64,64))
            img_array = np.array(img_resized).flatten()/255.0
            # PCA
            img_pca = pca.transform([img_array])[0]

            # Modelo Quântico
            pred_quantum = circuit_mera(model_weights_quantum, img_pca, quantum_gates)
            prob_quantum = (pred_quantum+1)/2
            class_quantum = 1 if prob_quantum>0.5 else 0
            conf_quantum = prob_quantum if class_quantum==1 else 1 - prob_quantum
            label_q = label_encoder.inverse_transform([class_quantum])[0]

            # Modelo Clássico (CNN)
            model_cnn_user = create_cnn_model((64,), num_classes)
            model_cnn_user.set_weights(global_weights_cnn)
            img_cnn = img_pca.reshape(1,64).reshape(-1,8,8,1)
            y_pred_probs_cnn = model_cnn_user.predict(img_cnn).flatten()
            y_pred_cnn = np.argmax(y_pred_probs_cnn)
            conf_cnn = y_pred_probs_cnn[y_pred_cnn]
            label_cnn = label_encoder.inverse_transform([y_pred_cnn])[0]

            print(f"\nImagem: {fn}")
            print("==> Modelo Quântico:")
            print(f"Classe Predita: {label_q}")
            print(f"Grau de Confiança: {conf_quantum:.4f}")

            print("==> Modelo Clássico (CNN):")
            print(f"Classe Predita: {label_cnn}")
            print(f"Grau de Confiança: {conf_cnn:.4f}")

            # Grad-CAM via tf-keras-vis
            try:
                conv_layer = None
                for layer in reversed(model_cnn_user.layers):
                    if isinstance(layer, layers.Conv2D):
                        conv_layer = layer
                        break
                if conv_layer is None:
                    raise ValueError("Nenhuma camada convolucional encontrada para Grad-CAM.")

                score_cnn = CategoricalScore(y_pred_cnn)
                gradcam = Gradcam(model_cnn_user, clone=False)
                heatmap = gradcam(score_cnn, [img_cnn], penultimate_layer=conv_layer.name)[0]

                plt.figure(figsize=(12,6))
                plt.subplot(1,2,1)
                plt.imshow(img_resized)
                plt.title("Imagem Original")
                plt.axis("off")

                plt.subplot(1,2,2)
                plt.imshow(img_resized, alpha=0.6)
                plt.imshow(heatmap, cmap='jet', alpha=0.4)
                plt.title(f"Grad-CAM - CNN\nClasse: {label_cnn} | Confiança: {conf_cnn:.2f}")
                plt.axis("off")

                gradcam_path = os.path.join(save_dir, f"user_image_gradcam_{fn}")
                plt.savefig(gradcam_path)
                plt.show()
                logging.info(f"Grad-CAM salvo em: {gradcam_path}")
            except Exception as e:
                logging.error(f"Erro ao gerar Grad-CAM: {e}")

    # Se desejar classificar imagens do usuário (upload)
    classify_user_image_both_models(global_weights, pca, label_encoder, global_weights_cnn)

    # =============================================================================
    # 15. Testes Unitários de Exemplo
    # =============================================================================

    def test_federated_averaging():
        """Exemplo de teste unitário para federated_averaging."""
        global_w = np.array([[[0.1, 0.2],[0.3,0.4]], [[0.5,0.6],[0.7,0.8]]])
        client_w = [
            np.array([[[0.11,0.21],[0.31,0.41]], [[0.51,0.61],[0.71,0.81]]]),
            np.array([[[0.12,0.22],[0.32,0.42]], [[0.52,0.62],[0.72,0.82]]])
        ]
        expected = np.array([[[0.115,0.215],[0.315,0.415]], [[0.515,0.615],[0.715,0.815]]])
        avg_w = federated_averaging(global_w, client_w)
        assert np.allclose(avg_w, expected), "federated_averaging falhou no teste unitário!"
        logging.info("Teste de federated_averaging passou com sucesso.")

    test_federated_averaging()
    logging.info("Processo finalizado com sucesso.")

else:
    logging.error("Encerrando o processo devido à falta do arquivo ZIP.")
